In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("../../data/clean/mbti_4.csv")
data.dropna(inplace=True)
data

In [ ]:
#train / test split and feature / target selection
train, test = train_test_split(data, test_size = 0.1, random_state=88, stratify=data.type)
features = ['com_stmt', 'pos_stmt', 'neg_stmt', 'neu_stmt', 'word_count',
            'unique_count', 'emoji_count', 'qe_mark', 'ex_mark', 'el_mark',
            'upper_count', 'url_count', 'img_count']
X_train, y_train = train[features], train['is_E']
X_test, y_test = test[features], test['is_E']
X_train_2, X_test_2 = train.iloc[:, 7:], test.iloc[:, 7:]
cv = KFold(n_splits=5,random_state=88,shuffle=True)

I&E prediction:
Simple Support Vector Classifier has the highest accuracy of 0.841270

In [1]:
#Support Vector Classifier (simple version -- no grid search / no cv)
svc = SVC(C=0.1).fit(X_train_2, y_train)
y_pred_svc = svc.predict(X_test_2)
model_svc_acc = sum(y_pred_svc == y_test) / len(y_test)
print(f'SVC Test Accuracy: {model_svc_acc:.4f}')

NameError: ignored

Stochastic Gradient Descent has the second highest accuracy score of 0.791451

In [ ]:
##Grid-Search 5-Cross-Validation Stochastic Gradient Descent Classifier
sgdc = SGDClassifier(random_state=88)
sgdc_grid = {"loss": ["hinge"],
             "penalty": ["l1", "l2"],
             "alpha": np.linspace(0.0001,0.0005,5)}
sgdc_search = GridSearchCV(sgdc, param_grid=sgdc_grid, cv=cv, scoring='accuracy', n_jobs=-1).fit(X_train_2, y_train)
#y_pred_sgdc = sgdc.predict(X_test_2)
#model_sgdc_acc = sum(y_pred_sgdc == y_test) / len(y_test)
print(sgdc_search.best_score_)
sgdc_best = sgdc_search.best_estimator_

Adaboost; accuracy - 0.778

In [ ]:
#Adaboost Classifier (simple version, grid-search cv not runnable)
abc = AdaBoostClassifier(random_state=88).fit(X_train_2, y_train)
#abc_grid = {'base_estimator' : ['gini', 'entropy'],
#            'algorithm' : ['SAMME', 'SAMME.R'],
#            'n_estimators' : [1],
#            'learning_rate' : np.linspace(0.1, 1, 10)
#            }
#abc_search = GridSearchCV(abc, param_grid=abc_grid, cv=cv, scoring='accuracy', n_jobs=-1).fit(X_train_2, y_train)
y_pred_abc = abc.predict(X_test_2)
model_abc_acc = sum(y_pred_abc == y_test) / len(y_test)
print(f'ABC Test Accuracy: {model_abc_acc:.4f}')
#print(abc_search.best_score_)
#abc_best = abc_search.best_estimator_

Random Forest: accuracy 0.778

In [ ]:
#Simple Version Random Forest
rf = RandomForestClassifier(random_state=88).fit(X_train_2, y_train)
y_pred_rf = rf.predict(X_test_2)
model_rf_acc = sum(y_pred_rf == y_test) / len(y_test)
print(f'Random Forest Test Accuracy: {model_rf_acc:.4f}')